In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('daum_news_crawl.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2954 entries, 0 to 2953
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   platform          2954 non-null   object
 1   category          2954 non-null   object
 2   publisher         2954 non-null   object
 3   publication_date  2954 non-null   object
 4   title             2954 non-null   object
 5   content           2939 non-null   object
 6   editor            2908 non-null   object
 7   article_url       2954 non-null   object
 8   stk1_name         2954 non-null   object
 9   stk1_count        2954 non-null   int64 
 10  stk2_name         2954 non-null   object
 11  stk2_count        2954 non-null   int64 
 12  stk3_name         2954 non-null   object
 13  stk3_count        2954 non-null   int64 
 14  stk4_name         2954 non-null   object
 15  stk4_count        2954 non-null   int64 
 16  stk5_name         2954 non-null   object
 17  stk5_count    

In [11]:
df['stk1_count']

0        5
1       11
2        6
3        0
4        2
        ..
2949     0
2950     0
2951     0
2952     4
2953     0
Name: stk1_count, Length: 2954, dtype: int64

In [10]:
df['stk1_name']

0       추천해요
1       추천해요
2       추천해요
3       추천해요
4       추천해요
        ... 
2949    추천해요
2950    추천해요
2951    추천해요
2952    추천해요
2953    추천해요
Name: stk1_name, Length: 2954, dtype: object

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import json
from datetime import datetime, timedelta
from selenium.webdriver.chrome.options import Options  

In [36]:
# 카테고리와 URL 매핑을 위한 설정
category_mapping = {
    '259': '금융',
    '258': '증권',
    '261': '산업_재계',
    '771': '중기_벤처',
    '260': '부동산',
    '262': '글로벌경제',
    '310': '생활경제',
    '263': '경제_일반',

}

def get_category_from_url(url):
    """URL에서 카테고리 번호를 추출하고 카테고리를 반환하는 함수"""
    category_id = url.split('/')[-1].split('?')[0]  # URL의 마지막 부분에서 카테고리 번호 추출
    return category_mapping.get(category_id, '기타')  # 매핑되지 않는 경우 '기타'로 설정

def infinity_scroll(driver, pause_time=1):
    """페이지 끝까지 스크롤을 내리는 함수"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div > div.section_more > a"))
                )
                more_button.click()
                time.sleep(pause_time)
            except Exception as e:
                print(f"더보기 버튼 클릭 오류: {e}")
                break

        last_height = new_height

def extract_sticker_reactions(soup):
    """BeautifulSoup 객체에서 스티커 반응 데이터를 추출하는 함수"""
    reaction_data = {
        'stk1_name': 'N/A', 'stk1_count': 0,
        'stk2_name': 'N/A', 'stk2_count': 0,
        'stk3_name': 'N/A', 'stk3_count': 0,
        'stk4_name': 'N/A', 'stk4_count': 0,
        'stk5_name': 'N/A', 'stk5_count': 0
    }

    reaction_selectors = [
        ('useful', 'stk1_name', 'stk1_count'),
        ('wow', 'stk2_name', 'stk2_count'),
        ('touched', 'stk3_name', 'stk3_count'),
        ('analytical', 'stk4_name', 'stk4_count'),
        ('recommend', 'stk5_name', 'stk5_count')
    ]

    for reaction, name_key, count_key in reaction_selectors:
        # 스티커 데이터를 찾기 위한 요소 선택
        label = soup.select_one(f'li.u_likeit_list.{reaction} > a > span.u_likeit_list_name._label')
        count = soup.select_one(f'li.u_likeit_list.{reaction} > a > span.u_likeit_list_count._count')

        if label:
            reaction_data[name_key] = label.text.strip()  # 스티커 이름
        if count:
            reaction_data[count_key] = int(count.text.strip().replace(',', ''))  # 스티커 개수
    
    return reaction_data

# def extract_article_details(url):
#     """기사의 세부 정보와 스티커 반응을 추출하는 함수"""
#     try:
#         print(f"기사 상세 정보 추출 중: {url}")
#         driver.get(url)

#         # 요소가 표시될 때까지 대기 (presence 대신 visibility 사용)
#         try:
#             WebDriverWait(driver, 10).until(
#                 EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.media_end_head_headline'))
#             )
#         except Exception as e:
#             print(f"요소 대기 중 오류 발생: {e}")
#             return None  # 오류 발생 시 None 반환

#         # BeautifulSoup으로 HTML 파싱
#         soup_in = BeautifulSoup(driver.page_source, 'html.parser')
        
#         # 첫 번째 방식: CSS 선택자로 타이틀 찾기
#         title_element = soup_in.select_one('h2.media_end_head_headline')
        
#         # 두 번째 방식: h2 태그에서 타이틀 찾기 (백업)
#         if not title_element:
#             title_element = soup_in.find('h2')

#         title = title_element.text.strip() if title_element else '제목 오류'

#         if soup_in.find('em', {"class": "media_end_head_journalist_name"}):
#             author = soup_in.find('em', {"class": "media_end_head_journalist_name"}).text
#         elif soup_in.find('span', {"class": "byline_s"}):
#             author = soup_in.find('span', {"class": "byline_s"}).text[:3] + " 기자"
#         else:
#             author = '기자명 오류'

#         content = soup_in.find('article', {"id": "dic_area"}).text if soup_in.find('article', {"id": "dic_area"}) else '본문 오류'
#         date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}) else '날짜 오류'
#         mod_date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}) else '수정 날짜 없음'

#         # 스티커 반응 데이터 추출
#         sticker_data = extract_sticker_reactions(soup_in)

#         print(sticker_data)
#         return title, author, content, date, mod_date, sticker_data

#     except Exception as e:
#         print(f"기사 상세 정보 추출 오류: {e}")
#         return None  # 오류 발생 시 None 반환

def extract_article_details(url):
    """기사의 세부 정보와 스티커 반응을 추출하는 함수"""
    try:
        print(f"기사 상세 정보 추출 중: {url}")
        driver.get(url)

        # 요소가 표시될 때까지 대기 (presence 대신 visibility 사용)
        try:
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.media_end_head_headline'))
            )
        except Exception as e:
            print(f"요소 대기 중 오류 발생: {e}")
            return None  # 오류 발생 시 None 반환

        # 페이지가 완전히 로드될 때까지 대기
        time.sleep(3)  # 데이터 로딩이 완료될 때까지 대기 (필요시 조정)

        # BeautifulSoup으로 HTML 파싱
        soup_in = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 제목, 내용 등 추출
        title_element = soup_in.select_one('h2.media_end_head_headline')
        title = title_element.text.strip() if title_element else '제목 오류'

        author = soup_in.find('em', {"class": "media_end_head_journalist_name"}).text if soup_in.find('em', {"class": "media_end_head_journalist_name"}) else '기자명 오류'
        content = soup_in.find('article', {"id": "dic_area"}).text if soup_in.find('article', {"id": "dic_area"}) else '본문 오류'
        date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}) else '날짜 오류'
        mod_date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}) else '수정 날짜 없음'

        # 스티커 반응 데이터 추출
        sticker_data = extract_sticker_reactions(soup_in)

        print(sticker_data)
        return title, author, content, date, mod_date, sticker_data

    except Exception as e:
        print(f"기사 상세 정보 추출 오류: {e}")
        return None  # 오류 발생 시 None 반환
      


def extract_comments(article_url):
    """네이버 뉴스 기사 URL에서 댓글을 추출하는 함수"""
    news_id = article_url.split("/")[-1]
    press_id = article_url.split("/")[-2]
    news_object = f"news{press_id}%2C{news_id}"

    url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_economy_m1&pool=cbox5&_cv=20240826125754&lang=ko&country=KR&objectId={news_object}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=FAVORITE&includeAllStatus=true&_=1725273489605"

    custom_header = {
        "Referer": article_url,
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
    }

    response = requests.get(url, headers=custom_header)

    json_string = response.text.split("_callback(")[-1].strip(');')

    try:
        json_data = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        return []

    comments = []
    if 'result' in json_data and 'commentList' in json_data['result']:
        for comment in json_data['result']['commentList']:
            user_id_no = comment.get("userIdNo", "unknown")
            if user_id_no is not None:
                user_unique = user_id_no[:5]
            else:
                user_unique = "unknown"

            comments.append({
                "user_id": comment.get("userName"),
                "user_unique": user_unique,
                "comment": comment.get("contents"),
                "comment_date": comment.get("regTime"),
                "update_date": comment.get("modTime", "수정되지 않음")
            })
    return comments

def scrape_news(date_range, base_url, category):
    """네이버 뉴스 페이지에서 기사를 스크랩하고 댓글을 추출하는 함수"""
    
    for date in date_range:
        title_list = []
        content_list = []
        author_list = []
        date_list = []
        update_date_list = []
        publisher_list = []  # 발행자 리스트
        url_list = []
        comments_list = []
        category_list = []  # 카테고리 리스트
        sticker_data_list = []  # 스티커 반응 데이터를 저장할 리스트

        url = f"{base_url}?date={date.strftime('%Y%m%d')}"  # 날짜를 YYYYMMDD 형식으로 변환

        print(f"크롤링 중: {url}")
        driver.get(url)
        time.sleep(1)

        infinity_scroll(driver)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        links = soup.select("div.sa_text > a")
        publishers = soup.select("div.sa_text_press")  # 발행자 정보 추출

        for link, publisher in zip(links, publishers):
            article_url = link.get('href')
            publisher_name = publisher.text.strip()  # 발행자 이름 추출

            # 기사 상세 정보 추출
            article_details = extract_article_details(article_url)

            # 오류 발생 시 해당 기사 건너뛰기
            if article_details is None:
                print(f"건너뜀: {article_url}")
                continue

            title, author, content, pub_date, mod_date, sticker_data = article_details

            url_list.append(article_url)
            publisher_list.append(publisher_name)
            category_list.append(category)

            title_list.append(title)
            author_list.append(author)
            content_list.append(content)
            date_list.append(pub_date)
            update_date_list.append(mod_date)
            sticker_data_list.append(sticker_data)

            comments = extract_comments(article_url)
            comments_list.extend(comments)

            print(f"기사 URL: {article_url}, 제목: {title}, 발행자: {publisher_name}, 카테고리: {category}")

        scraped_data = {
            "title": title_list,
            "author": author_list,
            "content": content_list,
            "publication_date": date_list,
            "update_date": update_date_list,
            "article_url": url_list,
            "publisher": publisher_list,
            "category": category_list,
            "comments": comments_list,
            "stickers": sticker_data_list
        }

        # 날짜별로 CSV 저장
        save_to_csv(date.strftime('%Y%m%d'), category, scraped_data)



def generate_date_range(start_date, end_date):
    """시작 날짜와 끝 날짜 사이의 날짜 범위를 생성하는 함수"""
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    delta = timedelta(days=1)
    while start <= end:
        yield start
        start += delta

def save_to_csv(date, category, scraped_data):
    """날짜별로 스크래핑한 데이터를 CSV 파일로 저장하는 함수"""
    file_name_articles = f"{date}_articles_{category}.csv"
    file_name_comments = f"{date}_comments_{category}.csv"

    df_articles = pd.DataFrame({
        'platform': '네이버',
        'category': scraped_data['category'],
        'publisher': scraped_data['publisher'],
        'publication_date': scraped_data['publication_date'],
        'title': scraped_data['title'],
        'content': scraped_data['content'],
        'editor': scraped_data['author'],
        'article_url': scraped_data['article_url'],
        'update_date': scraped_data['update_date'],
        'stk1_name': [x['stk1_name'] for x in scraped_data['stickers']],
        'stk1_count': [x['stk1_count'] for x in scraped_data['stickers']],
        'stk2_name': [x['stk2_name'] for x in scraped_data['stickers']],
        'stk2_count': [x['stk2_count'] for x in scraped_data['stickers']],
        'stk3_name': [x['stk3_name'] for x in scraped_data['stickers']],
        'stk3_count': [x['stk3_count'] for x in scraped_data['stickers']],
        'stk4_name': [x['stk4_name'] for x in scraped_data['stickers']],
        'stk4_count': [x['stk4_count'] for x in scraped_data['stickers']],
        'stk5_name': [x['stk5_name'] for x in scraped_data['stickers']],
        'stk5_count': [x['stk5_count'] for x in scraped_data['stickers']]
    })

    df_comments = pd.DataFrame(scraped_data['comments'])

    df_articles.to_csv(file_name_articles, index=False)
    df_comments.to_csv(file_name_comments, index=False)

    print(f"날짜별로 '{file_name_articles}' 파일에 기사 데이터가 저장되었습니다.")
    print(f"날짜별로 '{file_name_comments}' 파일에 댓글 데이터가 저장되었습니다.")

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드로 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 스크래핑 실행
start_date = '20240801'
end_date = '20240831'
date_range = generate_date_range(start_date, end_date)  # 날짜 범위 설정

# 여러 카테고리 URL을 순차적으로 처리
base_urls = [
    {"url": "https://news.naver.com/breakingnews/section/101/258", "category": "증권"},  # 증권
    {"url": "https://news.naver.com/breakingnews/section/101/261", "category": "산업"},  # 산업
    {"url": "https://news.naver.com/breakingnews/section/101/263", "category": "경제"},   # 경제일반
    {"url": "https://news.naver.com/breakingnews/section/101/262", "category": "글로벌경제"}  # 글로벌 경제
]

for base_url_info in base_urls:
    base_url = base_url_info["url"]
    category = base_url_info["category"]

    print(f"카테고리 {category} 크롤링 시작...")

    # 스크래핑 시작 (날짜별로 저장)
    scrape_news(date_range, base_url, category)

# 크롬 드라이버 종료
driver.quit()

카테고리 증권 크롤링 시작...
크롤링 중: https://news.naver.com/breakingnews/section/101/258?date=20240801
더보기 버튼 클릭 오류: Message: 

기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/277/0005454126
{'stk1_name': '쏠쏠정보', 'stk1_count': 0, 'stk2_name': '흥미진진', 'stk2_count': 0, 'stk3_name': '공감백배', 'stk3_count': 0, 'stk4_name': '분석탁월', 'stk4_count': 0, 'stk5_name': '후속강추', 'stk5_count': 0}
기사 URL: https://n.news.naver.com/mnews/article/277/0005454126, 제목: 몬테네그로 항소법원, 권도형 한국 송환 결정, 발행자: 아시아경제, 카테고리: 증권
기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/277/0005454124
{'stk1_name': '쏠쏠정보', 'stk1_count': 0, 'stk2_name': '흥미진진', 'stk2_count': 0, 'stk3_name': '공감백배', 'stk3_count': 0, 'stk4_name': '분석탁월', 'stk4_count': 0, 'stk5_name': '후속강추', 'stk5_count': 0}
기사 URL: https://n.news.naver.com/mnews/article/277/0005454124, 제목: 뉴욕증시, 9월 금리인하 전망에 강세…실업수당 청구 1년래 최고, 발행자: 아시아경제, 카테고리: 증권
기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/009/0005344218
{'stk1_name': '쏠쏠정보', 'stk1_count': 0, 'stk2_name': '흥미진

ConnectionError: HTTPSConnectionPool(host='apis.naver.com', port=443): Max retries exceeded with url: /commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_economy_m1&pool=cbox5&_cv=20240826125754&lang=ko&country=KR&objectId=news421%2C0007703787&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=FAVORITE&includeAllStatus=true&_=1725273489605 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x300c76330>: Failed to establish a new connection: [Errno 51] Network is unreachable'))

# 한달 네이버 스티커 반응 크롤링

In [2]:
# 카테고리와 URL 매핑을 위한 설정
category_mapping = {
    '259': '금융',
    '258': '증권',
    '261': '산업_재계',
    '771': '중기_벤처',
    '260': '부동산',
    '262': '글로벌경제',
    '310': '생활경제',
    '263': '경제_일반',

}

def get_category_from_url(url):
    """URL에서 카테고리 번호를 추출하고 카테고리를 반환하는 함수"""
    category_id = url.split('/')[-1].split('?')[0]  # URL의 마지막 부분에서 카테고리 번호 추출
    return category_mapping.get(category_id, '기타')  # 매핑되지 않는 경우 '기타'로 설정

def infinity_scroll(driver, pause_time=1):
    """페이지 끝까지 스크롤을 내리는 함수"""
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            try:
                more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div > div.section_more > a"))
                )
                more_button.click()
                time.sleep(pause_time)
            except Exception as e:
                print(f"더보기 버튼 클릭 오류: {e}")
                break

        last_height = new_height

def extract_sticker_reactions(soup):
    """BeautifulSoup 객체에서 스티커 반응 데이터를 추출하는 함수"""
    reaction_data = {
        'stk1_name': 'N/A', 'stk1_count': 0,
        'stk2_name': 'N/A', 'stk2_count': 0,
        'stk3_name': 'N/A', 'stk3_count': 0,
        'stk4_name': 'N/A', 'stk4_count': 0,
        'stk5_name': 'N/A', 'stk5_count': 0
    }

    reaction_selectors = [
        ('useful', 'stk1_name', 'stk1_count'),
        ('wow', 'stk2_name', 'stk2_count'),
        ('touched', 'stk3_name', 'stk3_count'),
        ('analytical', 'stk4_name', 'stk4_count'),
        ('recommend', 'stk5_name', 'stk5_count')
    ]

    for reaction, name_key, count_key in reaction_selectors:
        # 스티커 데이터를 찾기 위한 요소 선택
        label = soup.select_one(f'li.u_likeit_list.{reaction} > a > span.u_likeit_list_name._label')
        count = soup.select_one(f'li.u_likeit_list.{reaction} > a > span.u_likeit_list_count._count')

        if label:
            reaction_data[name_key] = label.text.strip()  # 스티커 이름
        if count:
            reaction_data[count_key] = int(count.text.strip().replace(',', ''))  # 스티커 개수
    
    return reaction_data

# def extract_article_details(url):
#     """기사의 세부 정보와 스티커 반응을 추출하는 함수"""
#     try:
#         print(f"기사 상세 정보 추출 중: {url}")
#         driver.get(url)

#         # 요소가 표시될 때까지 대기 (presence 대신 visibility 사용)
#         try:
#             WebDriverWait(driver, 10).until(
#                 EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.media_end_head_headline'))
#             )
#         except Exception as e:
#             print(f"요소 대기 중 오류 발생: {e}")
#             return None  # 오류 발생 시 None 반환

#         # BeautifulSoup으로 HTML 파싱
#         soup_in = BeautifulSoup(driver.page_source, 'html.parser')
        
#         # 첫 번째 방식: CSS 선택자로 타이틀 찾기
#         title_element = soup_in.select_one('h2.media_end_head_headline')
        
#         # 두 번째 방식: h2 태그에서 타이틀 찾기 (백업)
#         if not title_element:
#             title_element = soup_in.find('h2')

#         title = title_element.text.strip() if title_element else '제목 오류'

#         if soup_in.find('em', {"class": "media_end_head_journalist_name"}):
#             author = soup_in.find('em', {"class": "media_end_head_journalist_name"}).text
#         elif soup_in.find('span', {"class": "byline_s"}):
#             author = soup_in.find('span', {"class": "byline_s"}).text[:3] + " 기자"
#         else:
#             author = '기자명 오류'

#         content = soup_in.find('article', {"id": "dic_area"}).text if soup_in.find('article', {"id": "dic_area"}) else '본문 오류'
#         date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}) else '날짜 오류'
#         mod_date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}) else '수정 날짜 없음'

#         # 스티커 반응 데이터 추출
#         sticker_data = extract_sticker_reactions(soup_in)

#         print(sticker_data)
#         return title, author, content, date, mod_date, sticker_data

#     except Exception as e:
#         print(f"기사 상세 정보 추출 오류: {e}")
#         return None  # 오류 발생 시 None 반환

def extract_article_details(url):
    """기사의 세부 정보와 스티커 반응을 추출하는 함수"""
    try:
        print(f"기사 상세 정보 추출 중: {url}")
        driver.get(url)

        # 요소가 표시될 때까지 대기 (presence 대신 visibility 사용)
        try:
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.media_end_head_headline'))
            )
        except Exception as e:
            print(f"요소 대기 중 오류 발생: {e}")
            return None  # 오류 발생 시 None 반환

        # 페이지가 완전히 로드될 때까지 대기
        time.sleep(3)  # 데이터 로딩이 완료될 때까지 대기 (필요시 조정)

        # BeautifulSoup으로 HTML 파싱
        soup_in = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 제목, 내용 등 추출
        title_element = soup_in.select_one('h2.media_end_head_headline')
        title = title_element.text.strip() if title_element else '제목 오류'

        author = soup_in.find('em', {"class": "media_end_head_journalist_name"}).text if soup_in.find('em', {"class": "media_end_head_journalist_name"}) else '기자명 오류'
        content = soup_in.find('article', {"id": "dic_area"}).text if soup_in.find('article', {"id": "dic_area"}) else '본문 오류'
        date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}) else '날짜 오류'
        mod_date = soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}).text if soup_in.find('span', {"class": "media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME"}) else '수정 날짜 없음'

        # 스티커 반응 데이터 추출
        sticker_data = extract_sticker_reactions(soup_in)

        print(sticker_data)
        return title, author, content, date, mod_date, sticker_data

    except Exception as e:
        print(f"기사 상세 정보 추출 오류: {e}")
        return None  # 오류 발생 시 None 반환
      


def extract_comments(article_url):
    """네이버 뉴스 기사 URL에서 댓글을 추출하는 함수"""
    news_id = article_url.split("/")[-1]
    press_id = article_url.split("/")[-2]
    news_object = f"news{press_id}%2C{news_id}"

    url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_economy_m1&pool=cbox5&_cv=20240826125754&lang=ko&country=KR&objectId={news_object}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=FAVORITE&includeAllStatus=true&_=1725273489605"

    custom_header = {
        "Referer": article_url,
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
    }

    response = requests.get(url, headers=custom_header)

    json_string = response.text.split("_callback(")[-1].strip(');')

    try:
        json_data = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        return []

    comments = []
    if 'result' in json_data and 'commentList' in json_data['result']:
        for comment in json_data['result']['commentList']:
            user_id_no = comment.get("userIdNo", "unknown")
            if user_id_no is not None:
                user_unique = user_id_no[:5]
            else:
                user_unique = "unknown"

            comments.append({
                "user_id": comment.get("userName"),
                "user_unique": user_unique,
                "comment": comment.get("contents"),
                "comment_date": comment.get("regTime"),
                "update_date": comment.get("modTime", "수정되지 않음")
            })
    return comments

def scrape_news(date_range, base_url, category):
    """네이버 뉴스 페이지에서 기사를 스크랩하고 댓글을 추출하는 함수"""
    
    for date in date_range:
        title_list = []
        content_list = []
        author_list = []
        date_list = []
        update_date_list = []
        publisher_list = []  # 발행자 리스트
        url_list = []
        comments_list = []
        category_list = []  # 카테고리 리스트
        sticker_data_list = []  # 스티커 반응 데이터를 저장할 리스트

        url = f"{base_url}?date={date.strftime('%Y%m%d')}"  # 날짜를 YYYYMMDD 형식으로 변환

        print(f"크롤링 중: {url}")
        driver.get(url)
        time.sleep(1)

        infinity_scroll(driver)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        links = soup.select("div.sa_text > a")
        publishers = soup.select("div.sa_text_press")  # 발행자 정보 추출

        for link, publisher in zip(links, publishers):
            article_url = link.get('href')
            publisher_name = publisher.text.strip()  # 발행자 이름 추출

            # 기사 상세 정보 추출
            article_details = extract_article_details(article_url)

            # 오류 발생 시 해당 기사 건너뛰기
            if article_details is None:
                print(f"건너뜀: {article_url}")
                continue

            title, author, content, pub_date, mod_date, sticker_data = article_details

            url_list.append(article_url)
            publisher_list.append(publisher_name)
            category_list.append(category)

            title_list.append(title)
            author_list.append(author)
            content_list.append(content)
            date_list.append(pub_date)
            update_date_list.append(mod_date)
            sticker_data_list.append(sticker_data)

            comments = extract_comments(article_url)
            comments_list.extend(comments)

            print(f"기사 URL: {article_url}, 제목: {title}, 발행자: {publisher_name}, 카테고리: {category}")

        scraped_data = {
            "title": title_list,
            "author": author_list,
            "content": content_list,
            "publication_date": date_list,
            "update_date": update_date_list,
            "article_url": url_list,
            "publisher": publisher_list,
            "category": category_list,
            "comments": comments_list,
            "stickers": sticker_data_list
        }

        # 날짜별로 CSV 저장
        save_to_csv(date.strftime('%Y%m%d'), category, scraped_data)



def generate_date_range(start_date, end_date):
    """시작 날짜와 끝 날짜 사이의 날짜 범위를 생성하는 함수"""
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    delta = timedelta(days=1)
    while start <= end:
        yield start
        start += delta

def save_to_csv(date, category, scraped_data):
    """날짜별로 스크래핑한 데이터를 CSV 파일로 저장하는 함수"""
    file_name_articles = f"{date}_articles_{category}.csv"
    file_name_comments = f"{date}_comments_{category}.csv"

    df_articles = pd.DataFrame({
        'platform': '네이버',
        'category': scraped_data['category'],
        # 'publisher': scraped_data['publisher'],
        'publication_date': scraped_data['publication_date'],
        'title': scraped_data['title'],
        # 'content': scraped_data['content'],
        # 'editor': scraped_data['author'],
        'article_url': scraped_data['article_url'],
        'update_date': scraped_data['update_date'],
        'stk1_name': [x['stk1_name'] for x in scraped_data['stickers']],
        'stk1_count': [x['stk1_count'] for x in scraped_data['stickers']],
        'stk2_name': [x['stk2_name'] for x in scraped_data['stickers']],
        'stk2_count': [x['stk2_count'] for x in scraped_data['stickers']],
        'stk3_name': [x['stk3_name'] for x in scraped_data['stickers']],
        'stk3_count': [x['stk3_count'] for x in scraped_data['stickers']],
        'stk4_name': [x['stk4_name'] for x in scraped_data['stickers']],
        'stk4_count': [x['stk4_count'] for x in scraped_data['stickers']],
        'stk5_name': [x['stk5_name'] for x in scraped_data['stickers']],
        'stk5_count': [x['stk5_count'] for x in scraped_data['stickers']]
    })

    df_comments = pd.DataFrame(scraped_data['comments'])

    df_articles.to_csv(file_name_articles, index=False)
    df_comments.to_csv(file_name_comments, index=False)

    print(f"날짜별로 '{file_name_articles}' 파일에 기사 데이터가 저장되었습니다.")
    print(f"날짜별로 '{file_name_comments}' 파일에 댓글 데이터가 저장되었습니다.")

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드로 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 스크래핑 실행
start_date = '20240801'
end_date = '20240831'
date_range = generate_date_range(start_date, end_date)  # 날짜 범위 설정

# 여러 카테고리 URL을 순차적으로 처리
base_urls = [
    {"url": "https://news.naver.com/breakingnews/section/101/259", "category": "금융"}, 
    {"url": "https://news.naver.com/breakingnews/section/101/258", "category": "증권"},  # 증권
    {"url": "https://news.naver.com/breakingnews/section/101/261", "category": "산업_재계"},  # 산업
    {"url": "https://news.naver.com/breakingnews/section/101/771", "category": "중기_벤처"}, 
    {"url": "https://news.naver.com/breakingnews/section/101/260", "category": "부동산"}, 
    {"url": "https://news.naver.com/breakingnews/section/101/262", "category": "글로벌경제"}, 
    {"url": "https://news.naver.com/breakingnews/section/101/310", "category": "생활경제"},   # 경제일반
    {"url": "https://news.naver.com/breakingnews/section/101/263", "category": "경제_일반"}  # 글로벌 경제
]

for base_url_info in base_urls:
    base_url = base_url_info["url"]
    category = base_url_info["category"]

    print(f"카테고리 {category} 크롤링 시작...")

    # 스크래핑 시작 (날짜별로 저장)
    scrape_news(date_range, base_url, category)

# 크롬 드라이버 종료
driver.quit()

카테고리 금융 크롤링 시작...
크롤링 중: https://news.naver.com/breakingnews/section/101/259?date=20240801
더보기 버튼 클릭 오류: Message: 

기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/082/0001282255
{'stk1_name': '쏠쏠정보', 'stk1_count': 3, 'stk2_name': '흥미진진', 'stk2_count': 0, 'stk3_name': '공감백배', 'stk3_count': 2, 'stk4_name': '분석탁월', 'stk4_count': 0, 'stk5_name': '후속강추', 'stk5_count': 1}
기사 URL: https://n.news.naver.com/mnews/article/082/0001282255, 제목: [속보] 몬테네그로 항소법원 “권도형, 한국으로 송환해야”, 발행자: 부산일보, 카테고리: 금융
기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/421/0007704171
{'stk1_name': '쏠쏠정보', 'stk1_count': 1, 'stk2_name': '흥미진진', 'stk2_count': 0, 'stk3_name': '공감백배', 'stk3_count': 0, 'stk4_name': '분석탁월', 'stk4_count': 0, 'stk5_name': '후속강추', 'stk5_count': 0}
기사 URL: https://n.news.naver.com/mnews/article/421/0007704171, 제목: WB '중진국 함정' 극복 대표 사례로 韓 꼽아…"개도국의 필독서", 발행자: 뉴스1, 카테고리: 금융
기사 상세 정보 추출 중: https://n.news.naver.com/mnews/article/014/0005222263
{'stk1_name': '쏠쏠정보', 'stk1_count': 3, 'stk2_name

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=128.0.6613.120)
Stacktrace:
0   chromedriver                        0x0000000100aaf998 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000100aa7e00 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x00000001006acbe0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100696a68 core::str::slice_error_fail::h6c488016ada29016 + 64036
4   chromedriver                        0x00000001006969a8 core::str::slice_error_fail::h6c488016ada29016 + 63844
5   chromedriver                        0x0000000100729f08 cxxbridge1$string$len + 602316
6   chromedriver                        0x00000001006e5a08 cxxbridge1$string$len + 322508
7   chromedriver                        0x00000001006e666c cxxbridge1$string$len + 325680
8   chromedriver                  